# Feature Engineering. Практическая работа

Сдаётся на проверку.

## Цели практической работы

Научиться: 

*   генерировать новые признаки; 
*   нормализовать и кодировать данные с помощью lambda-функций и библиотеки scikit-learn.


## Что входит в практическую работу

1. Построить признаки из строк.
2. Преобразовать категориальные переменные.
3. Стандартизировать данные.
4. Использовать MinMaxScaler для нормализации данных.
5. Создать признаки на основе дат.
6. Сохранить датафрейм для этапа моделирования.

## Что оценивается
- Программа выдаёт верный ответ на заданном наборе данных.
- Описаны причины выбранного решения, если требуется.
- Код читабелен: переменным даны осмысленные названия, соблюдены отступы и правила расстановки пробелов.
- При построении графиков название и оси подписаны, выведена легенда, если требуется. 
- Репозиторий проекта содержит осмысленные коммиты, содержащие конкретные реализованные фичи, ветки названы согласно назначению, лишние файлы не хранятся в репозитории.
- В репозитории проекта соблюдена иерархия директорий согласно списку модулей и содержащихся в них тем.
- Репозиторий проекта содержит файлы с данными, полученными в результате выполнения практической работы.


## Как отправить работу на проверку
Сдайте практическую работу этого модуля через систему контроля версий Git сервиса Skillbox GitLab. После загрузки работы на проверку напишите об этом в личном кабинете своему проверяющему куратору.


## Задача

Мы продолжим исследовать и готовить данные для этапа моделирования.

Вам предстоит работать с небольшой выборкой из коллекции подержанных автомобилей, выставленных на продажу в Соединённых Штатах, представленной в файле `data/vehicles_dataset_upd.csv`. На этих данных вы построите первую модель классификации, определяющую категорию цены подержанного автомобиля в зависимости от характеристик транспортного средства.

В этой практической работе вы продолжите генерировать дополнительные признаки и завершите подготовку датасета для этапа моделирования.

## Описание датасета:
- `id`— идентификатор записи;
- `url`— URL-записи о продаже;
- `region`— регион;
- `region_url`— URL региона;
- `price`— стоимость;
- `year`— год выпуска;
- `manufacturer`— производитель;
- `model`— модель;
- `condition`— состояние;
- `cylinders`— количество цилиндров;
- `fuel`— тип топлива;
- `odometer`— количество пройденных миль;
- `title_status`— статус;
- `transmission`— коробка передач;
- `VIN`— идентификационный номер;
- `drive`— тип привода;
- `size`— размер;
- `type`— кузов;
- `paint_color`— цвет;
- `image_url`— URL изображения;
- `description`— указанное описание;
- `county`— страна;
- `state`— штат;
- `lat`— широта;
- `long`— долгота;
- `posting_date`— дата размещения объявления о продаже;
- `price_category`— категория цены.


In [1]:
# Импортируйте необходимые библиотеки
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder

### Задача 1. Построение признаков из строк

1. Загрузите датасет из `data/vehicles_dataset_upd6.csv`, выведите первые пять записей и список колонок.

In [2]:
# Ваш код здесь
df = pd.read_csv('data/vehicles_dataset_upd6.csv')
print(df.columns)
df.head()

Index(['id', 'url', 'region', 'region_url', 'price', 'year', 'manufacturer',
       'model', 'fuel', 'odometer', 'title_status', 'transmission',
       'image_url', 'description', 'state', 'lat', 'long', 'posting_date',
       'price_category', 'date', 'odometer_km', 'odometer/price', 'region_new',
       'region_corrected', 'manufacturer_model', 'desc_len', 'model_in_desc',
       'price_k$', 'age_category', 'model_len', 'model_word_count', 'is_audi',
       'is_ford', 'is_chevrolet', 'is_toyota', 'is_jeep', 'is_honda',
       'is_nissan', 'x0_diesel', 'x0_electric', 'x0_gas', 'x0_hybrid',
       'x0_other', 'std_scaled_odometer', 'std_scaled_price'],
      dtype='object')


,id,url,region,region_url,price,year,manufacturer,model,fuel,odometer,...,is_jeep,is_honda,is_nissan,x0_diesel,x0_electric,x0_gas,x0_hybrid,x0_other,std_scaled_odometer,std_scaled_price
0,7308295377,https://chattanooga.craigslist.org/ctd/d/chatt...,chattanooga,https://chattanooga.craigslist.org,54990,2020,ram,2500 crew cab big horn,diesel,27442,...,0,0,0,1.0,0.0,0.0,0.0,0.0,-1.079390,2.958509
1,7316380095,https://newjersey.craigslist.org/ctd/d/carlsta...,north jersey,https://newjersey.craigslist.org,16942,2016,ford,explorer 4wd 4dr xlt,other,60023,...,0,0,0,0.0,0.0,0.0,0.0,1.0,-0.560115,-0.085826
2,7313733749,https://reno.craigslist.org/ctd/d/atlanta-2017...,reno / tahoe,https://reno.craigslist.org,35590,2017,volkswagen,golf r hatchback,gas,14048,...,0,0,0,0.0,0.0,1.0,0.0,0.0,-1.292863,1.406256
3,7308210929,https://fayetteville.craigslist.org/ctd/d/rale...,fayetteville,https://fayetteville.craigslist.org,14500,2013,toyota,rav4,gas,117291,...,0,0,0,0.0,0.0,1.0,0.0,0.0,0.352621,-0.281218
4,7303797340,https://knoxville.craigslist.org/ctd/d/knoxvil...,knoxville,https://knoxville.craigslist.org,14590,2012,bmw,1 series 128i coupe 2d,other,80465,...,0,0,0,0.0,0.0,0.0,0.0,1.0,-0.234311,-0.274017


2. Выведите список значений и количество уникальных значений для модели автомобиля.

In [3]:
df.model.nunique()

3341

In [4]:
# Ваш код здесь
df.model.to_list()

['2500 crew cab big horn',
 'explorer 4wd 4dr xlt',
 'golf r hatchback',
 'rav4',
 '1 series 128i coupe 2d',
 'fusion se',
 'accord',
 'charger',
 'altima',
 'galant',
 '500 pop',
 'town car',
 'e-series van universal by glaval designer luxury series conversion leather 16" dvd limo lighting bed',
 'g. caravan se',
 'silverado 2500hd 4x4',
 'mustang',
 'maxima sv',
 'civic',
 'explorer',
 'vue v6',
 'tahoe',
 'f-350',
 'f150',
 'clubman',
 'xts sedan 4d',
 '3500',
 '2500 4x4',
 'rogue',
 'silverado 1500',
 'compass 4x4',
 'camry solara 2dr cpe se v6 auto (natl)',
 'silverado',
 'renegade',
 'forte5',
 'f-150',
 'wrangler unlimited all new',
 'mazda3 touring sedan 4d',
 '535i',
 'park avenue',
 'focus',
 'camry',
 'silverado 1500',
 'gx',
 'f150 lariat',
 'titan',
 'silverado 1500',
 'corolla',
 'explorer',
 'colorado extended cab',
 'tiguan 2.0t s',
 'rav4',
 'impreza',
 'accord',
 'explorer',
 'jetta s',
 'super duty f-250 srw',
 'civic',
 'altima 2.5 s',
 'q5',
 'es 350 f sport sedan 

3. Количество уникальных значений велико. Попробуем их сократить, оставив только первое слово в наименовании модели. С помощью lambda-функции и преобразования над строками создайте новый признак `short_model`, содержащий только первое слово из наименования модели, выведите его и количество получившихся уникальных значений.

In [5]:
# Ваш код здесь
df['short_model']=df.model.apply(lambda x: x.split(' ')[0])
print(df['short_model'].nunique())


943


In [6]:
df.short_model.to_list()

['2500',
 'explorer',
 'golf',
 'rav4',
 '1',
 'fusion',
 'accord',
 'charger',
 'altima',
 'galant',
 '500',
 'town',
 'e-series',
 'g.',
 'silverado',
 'mustang',
 'maxima',
 'civic',
 'explorer',
 'vue',
 'tahoe',
 'f-350',
 'f150',
 'clubman',
 'xts',
 '3500',
 '2500',
 'rogue',
 'silverado',
 'compass',
 'camry',
 'silverado',
 'renegade',
 'forte5',
 'f-150',
 'wrangler',
 'mazda3',
 '535i',
 'park',
 'focus',
 'camry',
 'silverado',
 'gx',
 'f150',
 'titan',
 'silverado',
 'corolla',
 'explorer',
 'colorado',
 'tiguan',
 'rav4',
 'impreza',
 'accord',
 'explorer',
 'jetta',
 'super',
 'civic',
 'altima',
 'q5',
 'es',
 'lx450',
 'pilot',
 'Diahatsu',
 'other',
 'sequoia',
 'f-150',
 '1500',
 'civic',
 'tucson',
 'silverado',
 'sierra',
 'mazda3',
 'grand',
 'nautilus',
 'f-150',
 'elantra',
 'x1',
 'f-150',
 'fusion',
 'camaro',
 'accord',
 'explorer',
 'benz',
 'tacoma',
 'tsx',
 'sebring',
 'yukon',
 'cooper',
 'f-150',
 'enclave',
 'x4',
 'sequioa',
 'xts',
 'xc90',
 'is300',

### Задача 2. Преобразование категориальных переменных

1. Для категориальных переменных `short_model`, `transmission`, `region`, `manufacturer`, `state`, `title_status`, `age_category` распечатайте наименование категории и количество уникальных значений в ней.


In [7]:
# Ваш код здесь
print(df['short_model'].name,df.short_model.nunique(),"\n",df.transmission.name,df.transmission.nunique(),"\n", \
      df.region.name,df.region.nunique(),"\n", df.manufacturer.name,df.manufacturer.nunique(),"\n" \
     , df.state.name,df.state.nunique(),"\n", df.title_status.name,df.title_status.nunique(),"\n" \
     , df.age_category.name,df.age_category.nunique())


short_model 943 
 transmission 3 
 region 393 
 manufacturer 40 
 state 51 
 title_status 6 
 age_category 3


2. Создайте датафрейм `data`, содержащий только выбранные категориальные переменные.

In [8]:
# Ваш код здесь
data = df[['short_model', 'transmission', 'region', 'manufacturer', 'state', 'title_status', 'age_category']]
data

,short_model,transmission,region,manufacturer,state,title_status,age_category
0,2500,other,chattanooga,ram,tn,clean,new
1,explorer,automatic,north jersey,ford,nj,clean,new
2,golf,other,reno / tahoe,volkswagen,ca,clean,new
3,rav4,automatic,fayetteville,toyota,nc,clean,average
4,1,other,knoxville,bmw,tn,clean,average
...,...,...,...,...,...,...,...
9614,rav4,automatic,chautauqua,toyota,ny,clean,old
9615,wrangler,other,binghamton,jeep,ny,clean,average
9616,a3,automatic,salem,audi,or,clean,average
9617,cayenne,automatic,madison,porsche,wi,clean,new


3. Создайте объект класса OneHotEncoder, который будет отвечать за преобразование. Уточните, что на выходе должна быть неразреженная матрица.

In [9]:
# Ваш код здесь
ohe = OneHotEncoder (sparse=False)

4. Идентифицируйте параметры преобразования на датафрейме с категориальными переменными.

In [10]:
# Ваш код здесь
ohe.fit(df[['short_model', 'transmission', 'region', 'manufacturer', 'state', 'title_status', 'age_category']])

C:\ProgramData\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


OneHotEncoder(sparse=False, sparse_output=False)

5. Примените преобразование для датафрейма с категориальными переменными и сохраните в новую переменную, распечатайте получившуюся матрицу и её размерность.

In [11]:
# Ваш код здесь
ohe_categories=ohe.transform(df[['short_model', 'transmission', 'region', 'manufacturer', 'state', 'title_status', 'age_category']])
ohe_categories

array([[0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 1., 0.]])

6. Выведите новые наименования признаков с помощью метода `get_feature_names_out`.

In [12]:
# Ваш код здесь
ohe.get_feature_names_out()

array(['short_model_-benz', 'short_model_1', 'short_model_124', ...,
       'age_category_average', 'age_category_new', 'age_category_old'],
      dtype=object)

7. Добавьте в исходный датафрейм получившиеся новые признаки, выведите первые пять записей датафрейма и список колонок.

In [13]:
# Ваш код здесь
df[ohe.get_feature_names_out()]=ohe_categories
df.head()

C:\Users\5800x-2\AppData\Local\Temp\ipykernel_21296\1434219075.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[ohe.get_feature_names_out()]=ohe_categories
C:\Users\5800x-2\AppData\Local\Temp\ipykernel_21296\1434219075.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[ohe.get_feature_names_out()]=ohe_categories
C:\Users\5800x-2\AppData\Local\Temp\ipykernel_21296\1434219075.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performa

C:\Users\5800x-2\AppData\Local\Temp\ipykernel_21296\1434219075.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[ohe.get_feature_names_out()]=ohe_categories
C:\Users\5800x-2\AppData\Local\Temp\ipykernel_21296\1434219075.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[ohe.get_feature_names_out()]=ohe_categories
C:\Users\5800x-2\AppData\Local\Temp\ipykernel_21296\1434219075.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performa

,id,url,region,region_url,price,year,manufacturer,model,fuel,odometer,...,state_wy,title_status_clean,title_status_lien,title_status_missing,title_status_parts only,title_status_rebuilt,title_status_salvage,age_category_average,age_category_new,age_category_old
0,7308295377,https://chattanooga.craigslist.org/ctd/d/chatt...,chattanooga,https://chattanooga.craigslist.org,54990,2020,ram,2500 crew cab big horn,diesel,27442,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,7316380095,https://newjersey.craigslist.org/ctd/d/carlsta...,north jersey,https://newjersey.craigslist.org,16942,2016,ford,explorer 4wd 4dr xlt,other,60023,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,7313733749,https://reno.craigslist.org/ctd/d/atlanta-2017...,reno / tahoe,https://reno.craigslist.org,35590,2017,volkswagen,golf r hatchback,gas,14048,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,7308210929,https://fayetteville.craigslist.org/ctd/d/rale...,fayetteville,https://fayetteville.craigslist.org,14500,2013,toyota,rav4,gas,117291,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,7303797340,https://knoxville.craigslist.org/ctd/d/knoxvil...,knoxville,https://knoxville.craigslist.org,14590,2012,bmw,1 series 128i coupe 2d,other,80465,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [14]:
df.columns

Index(['id', 'url', 'region', 'region_url', 'price', 'year', 'manufacturer',
       'model', 'fuel', 'odometer',
       ...
       'state_wy', 'title_status_clean', 'title_status_lien',
       'title_status_missing', 'title_status_parts only',
       'title_status_rebuilt', 'title_status_salvage', 'age_category_average',
       'age_category_new', 'age_category_old'],
      dtype='object', length=1485)

8. Посмотрите на результат обратного преобразования получившейся матрицы в исходные признаки, используйте `inverse_transform`.

In [15]:
# Ваш код здесь
ohe.inverse_transform(ohe_categories)

array([['2500', 'other', 'chattanooga', ..., 'tn', 'clean', 'new'],
       ['explorer', 'automatic', 'north jersey', ..., 'nj', 'clean',
        'new'],
       ['golf', 'other', 'reno / tahoe', ..., 'ca', 'clean', 'new'],
       ...,
       ['a3', 'automatic', 'salem', ..., 'or', 'clean', 'average'],
       ['cayenne', 'automatic', 'madison', ..., 'wi', 'clean', 'new'],
       ['1500', 'other', 'norfolk / hampton roads', ..., 'va', 'clean',
        'new']], dtype=object)

### Задача 3. Стандартизация данных

1. Создайте переменную под количественные переменные `lat`, `long`, `year`, `odometer/price`, `desc_len`, `model_in_desc`, `model_len`, `model_word_count` и создайте датафрейм `data`, содержащий только выбранные количественные переменные.


In [16]:
# Ваш код здесь
quantities = df[['lat', 'long', 'year', 'odometer/price', 'desc_len', 'model_in_desc', 'model_len', 'model_word_count']]
data = quantities
data

,lat,long,year,odometer/price,desc_len,model_in_desc,model_len,model_word_count
0,35.060000,-85.250000,2020,0.499036,4482,0,22,5
1,40.821805,-74.061962,2016,3.542852,968,0,20,4
2,33.779214,-84.411811,2017,0.394718,4286,0,16,3
3,35.715954,-78.655304,2013,8.089034,3241,0,4,1
4,35.970000,-83.940000,2012,5.515079,4851,0,22,5
...,...,...,...,...,...,...,...,...
9614,42.123900,-79.189500,2002,33.370412,1710,0,4,1
9615,43.216990,-77.755610,2008,7.835323,948,0,8,1
9616,44.925908,-122.982753,2011,16.696387,909,0,25,5
9617,43.029559,-89.397796,2015,1.941787,3644,0,7,1


2. Создайте объект `std_scaler` класса StandardScaler с параметрами по умолчанию, который будет отвечать за стандартизацию.

In [17]:
# Ваш код здесь
from sklearn.preprocessing import StandardScaler
std_scaler=StandardScaler()

3. Идентифицируйте параметры преобразования на датафрейме с количественными переменными.

In [18]:
# Ваш код здесь
std_scaler.fit(quantities)

StandardScaler()

4. Примените преобразование для датафрейма с количественными переменными и сохраните в новую переменную, распечатайте получившуюся матрицу и её размерность.

In [19]:
# Ваш код здесь
std_scaled_quantities = std_scaler.transform(quantities)
std_scaled_quantities

array([[-0.61917248,  0.48424452,  1.32239376, ..., -0.15578836,
         1.16303238,  1.91066901],
       [ 0.38801429,  1.11079989,  0.69597272, ..., -0.15578836,
         0.93208734,  1.23579936],
       [-0.84305905,  0.531185  ,  0.85257798, ..., -0.15578836,
         0.47019727,  0.56092971],
       ...,
       [ 1.10542805, -1.62887477, -0.08705357, ..., -0.15578836,
         1.50944993,  1.91066901],
       [ 0.7739386 ,  0.25195859,  0.53936746, ..., -0.15578836,
        -0.56905538, -0.78880959],
       [-0.33773799,  0.98546471,  0.85257798, ..., -0.15578836,
         1.16303238,  1.91066901]])

In [20]:
std_scaled_quantities.shape

(9619, 8)

5. Создайте наименования новых признаков по шаблону: `lat_std` для `lat` и так далее.

In [31]:
# Ваш код здесь
quantities_renamed = ['lat_std', 'long_std', 'year_std', 'odometer/price_std', 'desc_len_std', 'model_in_desc_std', 'model_len_std', 'model_word_count_std']
quantities_renamed

['lat_std',
 'long_std',
 'year_std',
 'odometer/price_std',
 'desc_len_std',
 'model_in_desc_std',
 'model_len_std',
 'model_word_count_std']

6. Добавьте в исходный датафрейм получившиеся новые признаки, выведите первые пять записей датафрейма и список колонок.

In [71]:
# Ваш код здесь
df[quantities_renamed]=std_scaled_quantities
df

,id,url,region,region_url,price,year,manufacturer,model,fuel,odometer,...,lat_std,long_std,year_std,odometer/price_std,desc_len_std,model_in_desc_std,model_len_std,model_word_count_std,lat_mm,long_mm
0,7308295377,https://chattanooga.craigslist.org/ctd/d/chatt...,chattanooga,https://chattanooga.craigslist.org,54990,2020,ram,2500 crew cab big horn,diesel,27442,...,-0.619172,0.484245,1.322394,-0.510784,0.632075,-0.155788,1.163032,1.910669,0.341769,0.288699
1,7316380095,https://newjersey.craigslist.org/ctd/d/carlsta...,north jersey,https://newjersey.craigslist.org,16942,2016,ford,explorer 4wd 4dr xlt,other,60023,...,0.388014,1.110800,0.695973,-0.402947,-0.646781,-0.155788,0.932087,1.235799,0.468665,0.333055
2,7313733749,https://reno.craigslist.org/ctd/d/atlanta-2017...,reno / tahoe,https://reno.craigslist.org,35590,2017,volkswagen,golf r hatchback,gas,14048,...,-0.843059,0.531185,0.852578,-0.514480,0.560744,-0.155788,0.470197,0.560930,0.313561,0.292022
3,7308210929,https://fayetteville.craigslist.org/ctd/d/rale...,fayetteville,https://fayetteville.craigslist.org,14500,2013,toyota,rav4,gas,117291,...,-0.504509,0.853562,0.226157,-0.241883,0.180435,-0.155788,-0.915473,-0.788810,0.356215,0.314844
4,7303797340,https://knoxville.craigslist.org/ctd/d/knoxvil...,knoxville,https://knoxville.craigslist.org,14590,2012,bmw,1 series 128i coupe 2d,other,80465,...,-0.460101,0.557607,0.069552,-0.333074,0.766366,-0.155788,1.163032,1.910669,0.361810,0.293893
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9614,7304876387,https://chautauqua.craigslist.org/ctd/d/falcon...,chautauqua,https://chautauqua.craigslist.org,4495,2002,toyota,rav4,gas,150000,...,0.615626,0.823646,-1.496501,0.653795,-0.376744,-0.155788,-0.915473,-0.788810,0.497341,0.312727
9615,7316152972,https://binghamton.craigslist.org/ctd/d/roches...,binghamton,https://binghamton.craigslist.org,14495,2008,jeep,wrangler,gas,113573,...,0.806702,0.903947,-0.556869,-0.250872,-0.654060,-0.155788,-0.453583,-0.788810,0.521415,0.318411
9616,7310993818,https://salem.craigslist.org/ctd/d/salem-2011-...,salem,https://salem.craigslist.org,8995,2011,audi,a3 2.0t premium plus pzev,gas,150184,...,1.105428,-1.628875,-0.087054,0.063061,-0.668253,-0.155788,1.509450,1.910669,0.559052,0.139104
9617,7306637427,https://madison.craigslist.org/ctd/d/madison-2...,madison,https://madison.craigslist.org,31900,2015,porsche,cayenne,hybrid,61943,...,0.773939,0.251959,0.539367,-0.459670,0.327100,-0.155788,-0.569055,-0.788810,0.517287,0.272255


### Задача 4. MinMax-нормализация данных

1. Импортируйте MinMaxScaler из `sklearn.preprocessing`.

In [33]:
# Ваш код здесь
from sklearn.preprocessing import MinMaxScaler

2. Создайте датафрейм, содержащий переменные `lat` и `long`.

In [34]:
# Ваш код здесь
df_geoposition = df[['lat','long']]
df_geoposition

,lat,long
0,35.060000,-85.250000
1,40.821805,-74.061962
2,33.779214,-84.411811
3,35.715954,-78.655304
4,35.970000,-83.940000
...,...,...
9614,42.123900,-79.189500
9615,43.216990,-77.755610
9616,44.925908,-122.982753
9617,43.029559,-89.397796


3. Повторите действия пунктов 2–6 из задачи 3 для добавления в исходный датафрейм признаков `lat_mm` и `long_mm`. Выведите получившийся датафрейм.

In [40]:
# Ваш код здесь
minmax_scaler_geo=MinMaxScaler()
minmax_scaler_geo.fit(df[['lat','long']])
minmax_scaled_geo = minmax_scaler_geo.transform(df[['lat','long']])
minmax_scaled_geo
minmax_scaled_geo.shape
quantities_renamed_geo = ['lat_mm', 'long_mm']
quantities_renamed
df[quantities_renamed_geo]=minmax_scaled_geo
df

,id,url,region,region_url,price,year,manufacturer,model,fuel,odometer,...,lat_std,long_std,year_std,odometer/price_std,desc_len_std,model_in_desc_std,model_len_std,model_word_count_std,lat_mm,long_mm
0,7308295377,https://chattanooga.craigslist.org/ctd/d/chatt...,chattanooga,https://chattanooga.craigslist.org,54990,2020,ram,2500 crew cab big horn,diesel,27442,...,-0.619172,0.484245,1.322394,-0.510784,0.632075,-0.155788,1.163032,1.910669,0.341769,0.288699
1,7316380095,https://newjersey.craigslist.org/ctd/d/carlsta...,north jersey,https://newjersey.craigslist.org,16942,2016,ford,explorer 4wd 4dr xlt,other,60023,...,0.388014,1.110800,0.695973,-0.402947,-0.646781,-0.155788,0.932087,1.235799,0.468665,0.333055
2,7313733749,https://reno.craigslist.org/ctd/d/atlanta-2017...,reno / tahoe,https://reno.craigslist.org,35590,2017,volkswagen,golf r hatchback,gas,14048,...,-0.843059,0.531185,0.852578,-0.514480,0.560744,-0.155788,0.470197,0.560930,0.313561,0.292022
3,7308210929,https://fayetteville.craigslist.org/ctd/d/rale...,fayetteville,https://fayetteville.craigslist.org,14500,2013,toyota,rav4,gas,117291,...,-0.504509,0.853562,0.226157,-0.241883,0.180435,-0.155788,-0.915473,-0.788810,0.356215,0.314844
4,7303797340,https://knoxville.craigslist.org/ctd/d/knoxvil...,knoxville,https://knoxville.craigslist.org,14590,2012,bmw,1 series 128i coupe 2d,other,80465,...,-0.460101,0.557607,0.069552,-0.333074,0.766366,-0.155788,1.163032,1.910669,0.361810,0.293893
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9614,7304876387,https://chautauqua.craigslist.org/ctd/d/falcon...,chautauqua,https://chautauqua.craigslist.org,4495,2002,toyota,rav4,gas,150000,...,0.615626,0.823646,-1.496501,0.653795,-0.376744,-0.155788,-0.915473,-0.788810,0.497341,0.312727
9615,7316152972,https://binghamton.craigslist.org/ctd/d/roches...,binghamton,https://binghamton.craigslist.org,14495,2008,jeep,wrangler,gas,113573,...,0.806702,0.903947,-0.556869,-0.250872,-0.654060,-0.155788,-0.453583,-0.788810,0.521415,0.318411
9616,7310993818,https://salem.craigslist.org/ctd/d/salem-2011-...,salem,https://salem.craigslist.org,8995,2011,audi,a3 2.0t premium plus pzev,gas,150184,...,1.105428,-1.628875,-0.087054,0.063061,-0.668253,-0.155788,1.509450,1.910669,0.559052,0.139104
9617,7306637427,https://madison.craigslist.org/ctd/d/madison-2...,madison,https://madison.craigslist.org,31900,2015,porsche,cayenne,hybrid,61943,...,0.773939,0.251959,0.539367,-0.459670,0.327100,-0.155788,-0.569055,-0.788810,0.517287,0.272255


4. Создайте переменную под географические признаки `lat`, `lat_std`, `lat_mm`, `long`, `long_std`, `long_mm` и выведите их. Посмотрите на различия в значениях для исходных признаков и нормализованных с помощью стандартизации и MinMax-преобразования. 

In [42]:
# Ваш код здесь
geo = df[['lat','lat_std', 'lat_mm', 'long', 'long_std', 'long_mm']]
geo

,lat,lat_std,lat_mm,long,long_std,long_mm
0,35.060000,-0.619172,0.341769,-85.250000,0.484245,0.288699
1,40.821805,0.388014,0.468665,-74.061962,1.110800,0.333055
2,33.779214,-0.843059,0.313561,-84.411811,0.531185,0.292022
3,35.715954,-0.504509,0.356215,-78.655304,0.853562,0.314844
4,35.970000,-0.460101,0.361810,-83.940000,0.557607,0.293893
...,...,...,...,...,...,...
9614,42.123900,0.615626,0.497341,-79.189500,0.823646,0.312727
9615,43.216990,0.806702,0.521415,-77.755610,0.903947,0.318411
9616,44.925908,1.105428,0.559052,-122.982753,-1.628875,0.139104
9617,43.029559,0.773939,0.517287,-89.397796,0.251959,0.272255


5. Выведите описательные статистики для всех географических признаков. Посмотрите на различия в статистиках для исходных признаков и нормализованных с помощью стандартизации и MinMax-преобразования.

In [43]:
# Ваш код здесь
geo.describe()

,lat,lat_std,lat_mm,long,long_std,long_mm
count,9619.000000,9.619000e+03,9619.000000,9619.000000,9.619000e+03,9619.000000
mean,38.602095,2.105257e-16,0.419779,-93.896875,-4.498602e-16,0.254418
std,5.720989,1.000052e+00,0.125997,17.857352,1.000052e+00,0.070797
min,19.541726,-3.331829e+00,0.000000,-158.069300,-3.593801e+00,0.000000
25%,34.920000,-6.436450e-01,0.338685,-106.944340,-7.306875e-01,0.202690
50%,39.300000,1.219966e-01,0.435149,-87.845800,3.388739e-01,0.278408
75%,42.382100,6.607601e-01,0.503028,-80.678711,7.402470e-01,0.306822
max,64.947500,4.605283e+00,1.000000,94.163200,1.053179e+01,1.000000


6. Выведите матрицу корреляций для всех географических признаков, чтобы убедиться, что нормализация признаков сохранила свойства исходных признаков.

In [44]:
# Ваш код здесь
geo.corr()

,lat,lat_std,lat_mm,long,long_std,long_mm
lat,1.000000,1.000000,1.000000,-0.134596,-0.134596,-0.134596
lat_std,1.000000,1.000000,1.000000,-0.134596,-0.134596,-0.134596
lat_mm,1.000000,1.000000,1.000000,-0.134596,-0.134596,-0.134596
long,-0.134596,-0.134596,-0.134596,1.000000,1.000000,1.000000
long_std,-0.134596,-0.134596,-0.134596,1.000000,1.000000,1.000000
long_mm,-0.134596,-0.134596,-0.134596,1.000000,1.000000,1.000000


### Задача 5. Признаки на основе дат

1. Выведите по отдельности колонки `posting_date` и `date`. 

In [49]:
# Ваш код здесьd
print(df.posting_date,df.date)

0       2021-04-17T12:30:50-0400
1       2021-05-03T15:40:21-0400
2       2021-04-28T03:52:20-0700
3       2021-04-17T10:08:57-0400
4       2021-04-08T15:10:56-0400
                  ...           
9614    2021-04-10T16:33:57-0400
9615    2021-05-03T09:36:30-0400
9616    2021-04-22T12:14:01-0700
9617    2021-04-14T09:14:42-0500
9618    2021-04-24T13:50:49-0400
Name: posting_date, Length: 9619, dtype: object 0       2021-04-17 16:30:50+00:00
1       2021-05-03 19:40:21+00:00
2       2021-04-28 10:52:20+00:00
3       2021-04-17 14:08:57+00:00
4       2021-04-08 19:10:56+00:00
                  ...            
9614    2021-04-10 20:33:57+00:00
9615    2021-05-03 13:36:30+00:00
9616    2021-04-22 19:14:01+00:00
9617    2021-04-14 14:14:42+00:00
9618    2021-04-24 17:50:49+00:00
Name: date, Length: 9619, dtype: object


2. Ранее преобразованная колонка `date` после сохранения в файл и загрузки из него содержит данные в строковом формате, поэтому измените её тип c помощью `pd.to_datetime`.

In [106]:
# Ваш код здесь
df['date'] = pd.to_datetime(df.date,utc=True)
df['date']

0      2021-04-17 16:30:50+00:00
1      2021-05-03 19:40:21+00:00
2      2021-04-28 10:52:20+00:00
3      2021-04-17 14:08:57+00:00
4      2021-04-08 19:10:56+00:00
                  ...           
9614   2021-04-10 20:33:57+00:00
9615   2021-05-03 13:36:30+00:00
9616   2021-04-22 19:14:01+00:00
9617   2021-04-14 14:14:42+00:00
9618   2021-04-24 17:50:49+00:00
Name: date, Length: 9619, dtype: datetime64[ns, UTC]

3. Несколькими способами извлеките номер месяца продажи автомобиля.

3.1. Из переменной `posting_date` с помощью lambda-функции и операций над строками.

In [72]:
# Ваш код здесь
df.posting_date.apply(lambda x: x.split('-')[1])

0       04
1       05
2       04
3       04
4       04
        ..
9614    04
9615    05
9616    04
9617    04
9618    04
Name: posting_date, Length: 9619, dtype: object

3.2. Из переменной `date` с помощью lambda-функции и метода `month`.

In [113]:
# Ваш код здесь
#pd.to_datetime(df.date,utc=True)
#df.date.dtype
#month_number = pd.to_datetime(df.date,utc=True)
#df['month_number'][0].month
#month_number = pd.to_datetime(df.date,utc=True)
df.date.apply(lambda x: x.month)

0       4
1       5
2       4
3       4
4       4
       ..
9614    4
9615    5
9616    4
9617    4
9618    4
Name: date, Length: 9619, dtype: int64

3.3. Из переменной `date` с помощью `dt` и метода `month`.

In [120]:
# Ваш код здесь
df.date.dt.month
#Вопрос на задаче 5 пункт 3.3
#здесь немного непонятно, а что за метод dt? разве у нас метод month не должен работать с датой в колонке date?

0       4
1       5
2       4
3       4
4       4
       ..
9614    4
9615    5
9616    4
9617    4
9618    4
Name: date, Length: 9619, dtype: int32

4. Создайте любым из способов (3.2 или 3.3) признак `month`, обозначающий номер месяца размещения объявления о продаже автомобиля, и распечатайте его.

In [121]:
# Ваш код здесь
df['month'] = df.date.dt.month
df['month']

C:\Users\5800x-2\AppData\Local\Temp\ipykernel_21296\1293335994.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['month'] = df.date.dt.month


0       4
1       5
2       4
3       4
4       4
       ..
9614    4
9615    5
9616    4
9617    4
9618    4
Name: month, Length: 9619, dtype: int32

5. Создайте признак `dayofweek`, обозначающий день недели размещения объявления о продаже автомобиля, и распечатайте его.

In [131]:
# Ваш код здесь
df['dayofweek'] = df.date.dt.dayofweek
df['dayofweek']

C:\Users\5800x-2\AppData\Local\Temp\ipykernel_21296\3072497085.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['dayofweek'] = df.date.dt.dayofweek


0       5
1       0
2       2
3       5
4       3
       ..
9614    5
9615    0
9616    3
9617    2
9618    5
Name: dayofweek, Length: 9619, dtype: int32

6. Создайте признак `diff_years`, обозначающий количество лет между годом производства автомобиля и годом размещения объявления о продаже автомобиля, и распечатайте его. Дополнительные корректировки не применяйте.

In [141]:
# Ваш код здесь
df['diff_years'] = (df.posting_date.apply(lambda x: x.split('-')[0]).astype(int)-df.year.astype(int))
df.diff_years

0        1
1        5
2        4
3        8
4        9
        ..
9614    19
9615    13
9616    10
9617     6
9618     4
Name: diff_years, Length: 9619, dtype: int32

In [137]:
df[['year','posting_date']]

,year,posting_date
0,2020,2021-04-17T12:30:50-0400
1,2016,2021-05-03T15:40:21-0400
2,2017,2021-04-28T03:52:20-0700
3,2013,2021-04-17T10:08:57-0400
4,2012,2021-04-08T15:10:56-0400
...,...,...
9614,2002,2021-04-10T16:33:57-0400
9615,2008,2021-05-03T09:36:30-0400
9616,2011,2021-04-22T12:14:01-0700
9617,2015,2021-04-14T09:14:42-0500


7. Примените стандартизацию к новым признаками и сохраните результат в новые признаки вида `month_std`.

In [143]:
# Ваш код здесь
std_scaler=StandardScaler()
std_scaler.fit(df[['month','dayofweek','diff_years']])
std_scaled_times = std_scaler.transform(df[['month','dayofweek','diff_years']])
std_scaled_times


array([[-0.61584563,  1.12028423, -1.32239376],
       [ 1.62378353, -1.37497211, -0.69597272],
       [-0.61584563, -0.37686957, -0.85257798],
       ...,
       [-0.61584563,  0.1221817 ,  0.08705357],
       [-0.61584563, -0.37686957, -0.53936746],
       [-0.61584563,  1.12028423, -0.85257798]])

In [145]:
std_scaled_times.shape

(9619, 3)

In [146]:

times_renamed = ['month_std', 'dayofweek_std', 'diff_years_std']
times_renamed
df[times_renamed]=std_scaled_times
df

C:\Users\5800x-2\AppData\Local\Temp\ipykernel_21296\569308915.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[times_renamed]=std_scaled_times
C:\Users\5800x-2\AppData\Local\Temp\ipykernel_21296\569308915.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[times_renamed]=std_scaled_times
C:\Users\5800x-2\AppData\Local\Temp\ipykernel_21296\569308915.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all 

,id,url,region,region_url,price,year,manufacturer,model,fuel,odometer,...,model_len_std,model_word_count_std,lat_mm,long_mm,month,dayofweek,diff_years,month_std,dayofweek_std,diff_years_std
0,7308295377,https://chattanooga.craigslist.org/ctd/d/chatt...,chattanooga,https://chattanooga.craigslist.org,54990,2020,ram,2500 crew cab big horn,diesel,27442,...,1.163032,1.910669,0.341769,0.288699,4,5,1,-0.615846,1.120284,-1.322394
1,7316380095,https://newjersey.craigslist.org/ctd/d/carlsta...,north jersey,https://newjersey.craigslist.org,16942,2016,ford,explorer 4wd 4dr xlt,other,60023,...,0.932087,1.235799,0.468665,0.333055,5,0,5,1.623784,-1.374972,-0.695973
2,7313733749,https://reno.craigslist.org/ctd/d/atlanta-2017...,reno / tahoe,https://reno.craigslist.org,35590,2017,volkswagen,golf r hatchback,gas,14048,...,0.470197,0.560930,0.313561,0.292022,4,2,4,-0.615846,-0.376870,-0.852578
3,7308210929,https://fayetteville.craigslist.org/ctd/d/rale...,fayetteville,https://fayetteville.craigslist.org,14500,2013,toyota,rav4,gas,117291,...,-0.915473,-0.788810,0.356215,0.314844,4,5,8,-0.615846,1.120284,-0.226157
4,7303797340,https://knoxville.craigslist.org/ctd/d/knoxvil...,knoxville,https://knoxville.craigslist.org,14590,2012,bmw,1 series 128i coupe 2d,other,80465,...,1.163032,1.910669,0.361810,0.293893,4,3,9,-0.615846,0.122182,-0.069552
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9614,7304876387,https://chautauqua.craigslist.org/ctd/d/falcon...,chautauqua,https://chautauqua.craigslist.org,4495,2002,toyota,rav4,gas,150000,...,-0.915473,-0.788810,0.497341,0.312727,4,5,19,-0.615846,1.120284,1.496501
9615,7316152972,https://binghamton.craigslist.org/ctd/d/roches...,binghamton,https://binghamton.craigslist.org,14495,2008,jeep,wrangler,gas,113573,...,-0.453583,-0.788810,0.521415,0.318411,5,0,13,1.623784,-1.374972,0.556869
9616,7310993818,https://salem.craigslist.org/ctd/d/salem-2011-...,salem,https://salem.craigslist.org,8995,2011,audi,a3 2.0t premium plus pzev,gas,150184,...,1.509450,1.910669,0.559052,0.139104,4,3,10,-0.615846,0.122182,0.087054
9617,7306637427,https://madison.craigslist.org/ctd/d/madison-2...,madison,https://madison.craigslist.org,31900,2015,porsche,cayenne,hybrid,61943,...,-0.569055,-0.788810,0.517287,0.272255,4,2,6,-0.615846,-0.376870,-0.539367


### Задача 6. Сохранение датафрейма для этапа моделирования

1. Удалите колонки, которые были исходными для формирования признаков, а также новые признаки, которые были добавлены с целью обучения.  Сохраните новый датафрейм в переменную `df_prepared`.

In [147]:
# Допишите код

columns_for_drop = ['year', 'url', 'region', 'region_url', 'manufacturer',
                    'model', 'fuel', 'odometer', 'title_status', 'transmission',
                    'image_url', 'description', 'state', 'lat', 'long', 'posting_date',
                    'odometer_km', 'odometer/price', 'region_new', 'region_corrected', 'manufacturer_model',
                    'desc_len', 'model_in_desc', 'price_k$', 'age_category', 'model_len', 'model_word_count',
                    'short_model', 'lat_mm', 'long_mm', 'date', 'std_scaled_price',
                    'month', 'dayofweek', 'diff_years',
                    'odometer/price_std']
df_prepared = df.drop(columns = columns_for_drop)

2. Распечатайте оставшийся список колонок и размерность финального датафрейма.

In [151]:
# Ваш код здесь
df_prepared.columns

Index(['id', 'price', 'price_category', 'is_audi', 'is_ford', 'is_chevrolet',
       'is_toyota', 'is_jeep', 'is_honda', 'is_nissan',
       ...
       'lat_std', 'long_std', 'year_std', 'desc_len_std', 'model_in_desc_std',
       'model_len_std', 'model_word_count_std', 'month_std', 'dayofweek_std',
       'diff_years_std'],
      dtype='object', length=1465)

In [153]:
df_prepared.shape

(9619, 1465)

3. Запишите преобразованный датафрейм в файл `data/vehicles_dataset_prepared.csv`, индекс не сохраняйте.

In [154]:
# Ваш код здесь
df_prepared.to_csv('data/vehicles_dataset_prepared.csv',index=False);

## Описание преобразованного датасета:
- `id`— идентификатор записи;
- `is_manufacturer_name`— признак производителя автомобиля;

- `region_*`— регион;
- `x0_*`— тип топлива;
- `manufacturer_*`— производитель;
- `short_model_*`— сокращённая модель автомобиля;
- `title_status_*`— статус;
- `transmission_*`— коробка передач;
- `state_*`— штат;
- `age_category_*`— возрастная категория автомобиля;

- `std_scaled_odometer`— количество пройденных миль (после стандартизации);
- `year_std`— год выпуска (после стандартизации);
- `lat_std`— широта (после стандартизации);
- `long_std`— долгота (после стандартизации);
- `odometer/price_std`— отношение стоимости к пробегу автомобиля (после стандартизации);
- `desc_len_std`— количество символов в тексте объявления о продаже (после стандартизации);
- `model_in_desc_std`— количество наименований модели автомобиля в тексте объявления о продаже (после стандартизации);
- `model_len_std`— длина наименования автомобиля (после стандартизации);
- `model_word_count_std`— количество слов в наименовании автомобиля (после стандартизации);
- `month_std`— номер месяца размещения объявления о продаже автомобиля (после стандартизации);
- `dayofweek_std`— день недели размещения объявления о продаже автомобиля (после стандартизации);
- `diff_years_std`— количество лет между годом производства автомобиля и годом размещения объявления о продаже автомобиля (после стандартизации);

- `price`— стоимость;
- `price_category`– категория цены.